# implemented a parallel kmer counter but it behaves strangely and I don't know why 

In [8]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import gc
import timeit
import dask.dataframe as dd
from dask.multiprocessing import get
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, plot_confusion_matrix
from scipy import sparse
import seaborn as sn
from joblib import dump, load
import multiprocessing as mp
from pandarallel import pandarallel

%matplotlib inline

# Import and pre-process Data and Labels 

In [9]:
labels = []
def importLabels():
    with open('./Data/shortlabels.txt', 'r') as filehandle:
#     with open('./Data/labels.txt', 'r') as filehandle:


        for line in filehandle:
            # remove linebreak, if exists, which is the last character of the string
            if(line[-1] == "\n"):
                currentPlace = line[:-1]
            else:
                currentPlace = line[:]
            # add item to the list
            labels.append(currentPlace)
importLabels()

In [10]:
identifiers = [] # list
sequences = [] # list
# with open('./Data/sequences.fasta') as fasta_file:  # Will close handle cleanly
with open('./Data/short.fasta') as fasta_file:  # Will close handle cleanly

    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequences.append(str(seq_record.seq))
print(len(sequences))

1000


In [22]:
# kmrdic = pd.DataFrame()


def kmersPerSequenceDict(sequence, lower, upper):
    global output
    dic =  {}
    for size in range(lower, upper+1):
        for x in range(len(sequence) - size + 1):
            if not str(sequence[x:x+size].lower()) in dic:
                dic[str(sequence[x:x+size].lower())] = 1
            else:
                dic[str(sequence[x:x+size].lower())] += 1
#     display(dic)
#     return dic
    output.append(dic, ignore_index=True)
    print(output.shape[0])

#     print('type of dic :', type(dic))

# time required to process one sequence for kmers between 5-6
### Time took to create concatenated k-mers sequences:  0.04519199999776902

In [23]:
# t_pds = timeit.Timer(lambda: genKmersDictionary(sequences[0], 5, 6))
# print("Time took to create concatenated k-mers sequences: ",  t_pds.timeit(number=1))


In [24]:
# from threading import Thread
# output = pd.DataFrame()
lower = 2
upper = 2

# for sequence in sequences:
#     t = Thread(target=kmersPerSequenceDict, args=(sequence,lower, upper))
#     t.start()

In [25]:
from multiprocessing.dummy import Pool as ThreadPool
from itertools import repeat
from multiprocessing import Pool, freeze_support
from functools import partial

output = pd.DataFrame()
pool = ThreadPool()

# results = pool.map(kmersPerSequenceDict, sequences,args=(lower, upper))
def kmersPerDataset_parallel():
    global results
#     results = pool.starmap(kmersPerSequenceDict,zip(sequences, repeat(lower),repeat( upper)))
    pool.map(partial(kmersPerSequenceDict, lower=lower, upper=upper), sequences)#zip(sequences, repeat(lower),repeat( upper)))

    pool.close()
    pool.join()

In [26]:
# df = pd.concatenate([genKmers(sequences[i], 5, 6) for i in range(len(sequences))])

# lower = 4
# upper = 5
# def kmersPerDataset():
#     global output
#     for sequence in sequences:
#         dic = kmersPerSequenceDict(sequence, lower, upper)
#         output = output.append(dic, ignore_index=True)

In [27]:
results = pd.DataFrame()

In [28]:

t_pds = timeit.Timer(lambda: kmersPerDataset_parallel())
print("Time took to create a DF of all k-mers sequences with occurences: " +  str(t_pds.timeit(number=1)) + "seonds")

00
0
0
0
0
0
0
0

0
0
0
0
0
00

0
0
0
0
0
0
0
0
0
00
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0000
0



0
0
0000
0
0
0


0
00
0
0
0
0

0
000


0
0
000

0

00
0

0
0
0
00
0
0
0
0
00
0

0
0

0
0
0
0
0
0
0
0
0
00
0
0
0
0
0
0
0

0
0

00

0
00
000
0
0
0
0

0
0


0
00
0
0
0
0
00
00

0
0
0
0

0

000

00
0


000
0


0
000
0
00

00
000
0


0
0
00
0

0
0


0

00
0
000

0
0
00
0
0

0

00
0
00
0
0

0

00
0
0
0
0
0
0
0

0
00
0
000


00
0
0
0
0
0
0

0

0
0
00

00

00
0
0
0


0
0
0
00

0
0
0
00
0
00
0

000
0


0
0
0
0
0

00

0
00
0

0
0
0
0
0
0
00
00
0

0
000
0
0
0
0
0
00

0
0
00

0

0
0
0
0
00
0

0

00
0
00
0

0
0
0
0
0

0
0
00


0
0
0
0
00
0
0

0
0
0
0
0
0
0
000

0
0
0

0
0
0
0
0
0
00
0
00
0
0
0


0
0
00

00
0
0
00

00
0
0
0
0
0

0
0
000

0
0
0
0
0

0
00
0

0
0
00


000

00

0
0

0
0
00
00

0
0
0
0
0

000
0
0
0
0

0

0
0
0
0
0
0
0
0
0
0
0
0
0
0
00
0
0
0
0
00
0
0
0
0
00

0
0
0
000
00

0

0
0

0

0
0
0
00
000
0
0

0
0
0

0
0
00


0
00
0

0
0

00

0
0
0
0
0
0
0
0
0
0
0
0


In [29]:
display(output)

""


### Time took to create a DF of all (2k-mers sequences with occurences:  820.6407018 s

In [30]:
len(results)
# type(results)

0

In [21]:
# #just to verify that zip works 
# argus = zip(sequences, repeat(lower),repeat( upper))
# display(list(argus))